# Sparkify Project Workspace
This workspace contains a tiny subset (128MB) of the full dataset available (12GB). Feel free to use this workspace to build your project, or to explore a smaller subset with Spark before deploying your cluster on the cloud. Instructions for setting up your Spark cluster is included in the last lesson of the Extracurricular Spark Course content.

You can follow the steps below to guide your data analysis and model building portion of this project.

In [1]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import udf
from pyspark.sql.types import StringType
from pyspark.sql.types import IntegerType
from pyspark.sql.functions import desc
from pyspark.sql.functions import asc
from pyspark.sql.functions import sum as Fsum, isnan, to_date, to_timestamp, from_unixtime

import datetime

import numpy as np
import pandas as pd
%matplotlib inline
import matplotlib.pyplot as plt

import findspark
findspark.init()

In [2]:
spark = SparkSession \
    .builder \
    .appName("Wrangling Data") \
    .getOrCreate()

# Load and Clean Dataset
In this workspace, the mini-dataset file is `mini_sparkify_event_data.json`. Load and clean the dataset, checking for invalid or missing data - for example, records without userids or sessionids. 

In [3]:
path = "mini_sparkify_event_data.json"
df = spark.read.json(path)
df_original = df

In [4]:
df.printSchema()

root
 |-- artist: string (nullable = true)
 |-- auth: string (nullable = true)
 |-- firstName: string (nullable = true)
 |-- gender: string (nullable = true)
 |-- itemInSession: long (nullable = true)
 |-- lastName: string (nullable = true)
 |-- length: double (nullable = true)
 |-- level: string (nullable = true)
 |-- location: string (nullable = true)
 |-- method: string (nullable = true)
 |-- page: string (nullable = true)
 |-- registration: long (nullable = true)
 |-- sessionId: long (nullable = true)
 |-- song: string (nullable = true)
 |-- status: long (nullable = true)
 |-- ts: long (nullable = true)
 |-- userAgent: string (nullable = true)
 |-- userId: string (nullable = true)



In [5]:
# Easier visual exploration as Pandas data frame
df.limit(5).toPandas()

# Questions to explore:
# sessionId unique across users? (can two users have the same sessionId?)
# what is the column "method"

,artist,auth,firstName,gender,itemInSession,lastName,length,level,location,method,page,registration,sessionId,song,status,ts,userAgent,userId
0,Martha Tilston,Logged In,Colin,M,50,Freeman,277.89016,paid,"Bakersfield, CA",PUT,NextSong,1538173362000,29,Rockpools,200,1538352117000,Mozilla/5.0 (Windows NT 6.1; WOW64; rv:31.0) G...,30
1,Five Iron Frenzy,Logged In,Micah,M,79,Long,236.09424,free,"Boston-Cambridge-Newton, MA-NH",PUT,NextSong,1538331630000,8,Canada,200,1538352180000,"""Mozilla/5.0 (Windows NT 6.1; WOW64) AppleWebK...",9
2,Adam Lambert,Logged In,Colin,M,51,Freeman,282.82730,paid,"Bakersfield, CA",PUT,NextSong,1538173362000,29,Time For Miracles,200,1538352394000,Mozilla/5.0 (Windows NT 6.1; WOW64; rv:31.0) G...,30
3,Enigma,Logged In,Micah,M,80,Long,262.71302,free,"Boston-Cambridge-Newton, MA-NH",PUT,NextSong,1538331630000,8,Knocking On Forbidden Doors,200,1538352416000,"""Mozilla/5.0 (Windows NT 6.1; WOW64) AppleWebK...",9
4,Daft Punk,Logged In,Colin,M,52,Freeman,223.60771,paid,"Bakersfield, CA",PUT,NextSong,1538173362000,29,Harder Better Faster Stronger,200,1538352676000,Mozilla/5.0 (Windows NT 6.1; WOW64; rv:31.0) G...,30


In [6]:
# Which pages do users interact with?
df.select("page").dropDuplicates().sort("page").show()

df.select("auth").dropDuplicates().sort("auth").show()

+--------------------+
|                page|
+--------------------+
|               About|
|          Add Friend|
|     Add to Playlist|
|              Cancel|
|Cancellation Conf...|
|           Downgrade|
|               Error|
|                Help|
|                Home|
|               Login|
|              Logout|
|            NextSong|
|            Register|
|         Roll Advert|
|       Save Settings|
|            Settings|
|    Submit Downgrade|
| Submit Registration|
|      Submit Upgrade|
|         Thumbs Down|
+--------------------+
only showing top 20 rows

+----------+
|      auth|
+----------+
| Cancelled|
|     Guest|
| Logged In|
|Logged Out|
+----------+



In [9]:
# https://stackoverflow.com/questions/52319958/pyspark-select-the-distinct-values-from-each-column
import pyspark.sql.functions as f
# df.select(*[f.collect_set(c).alias(c) for c in df.columns]).toPandas()

In [10]:
# Distinct values per column
# https://stackoverflow.com/questions/40888946/spark-dataframe-count-distinct-values-of-every-column
from pyspark.sql.functions import col, countDistinct

distinct_value_count = df.agg(*(countDistinct(col(c)).alias(c) for c in df.columns)).toPandas()
distinct_value_count

,artist,auth,firstName,gender,itemInSession,lastName,length,level,location,method,page,registration,sessionId,song,status,ts,userAgent,userId
0,17655,4,189,2,1322,173,14865,2,114,2,22,225,2354,58480,3,277447,56,226


In [ ]:
distinct_value_count > 2

,artist,auth,firstName,gender,itemInSession,lastName,length,level,location,method,page,registration,sessionId,song,status,ts,userAgent,userId
0,True,True,True,False,True,True,True,False,True,False,True,True,True,True,True,True,True,True


In [11]:
get_hour = udf(lambda x: datetime.datetime.fromtimestamp(x / 1000.0). hour)
# get_date = udf(lambda x: datetime.datetime.fromtimestamp(x / 1000.0). date)


In [12]:
df.head()

Row(artist='Martha Tilston', auth='Logged In', firstName='Colin', gender='M', itemInSession=50, lastName='Freeman', length=277.89016, level='paid', location='Bakersfield, CA', method='PUT', page='NextSong', registration=1538173362000, sessionId=29, song='Rockpools', status=200, ts=1538352117000, userAgent='Mozilla/5.0 (Windows NT 6.1; WOW64; rv:31.0) Gecko/20100101 Firefox/31.0', userId='30')

In [13]:
# https://stackoverflow.com/questions/48229043/python-pyspark-count-null-empty-and-nan
df.filter((df["ts"] == "") | df["ts"].isNull() | isnan(df["ts"])).count()

0

In [40]:
from pyspark.sql.types import LongType, TimestampType
from pyspark.sql.functions import substring, from_unixtime

df = df_original
df.printSchema()

root
 |-- artist: string (nullable = true)
 |-- auth: string (nullable = true)
 |-- firstName: string (nullable = true)
 |-- gender: string (nullable = true)
 |-- itemInSession: long (nullable = true)
 |-- lastName: string (nullable = true)
 |-- length: double (nullable = true)
 |-- level: string (nullable = true)
 |-- location: string (nullable = true)
 |-- method: string (nullable = true)
 |-- page: string (nullable = true)
 |-- registration: long (nullable = true)
 |-- sessionId: long (nullable = true)
 |-- song: string (nullable = true)
 |-- status: long (nullable = true)
 |-- ts: long (nullable = true)
 |-- userAgent: string (nullable = true)
 |-- userId: string (nullable = true)



In [42]:
df = df.withColumn("ts_new", substring(df.ts, 0, 10))
df = df.withColumn("ts_new_2", df.ts_new.cast(TimestampType()))

df = df.withColumn("ts_unix", from_unixtime(df.ts))
df = df.withColumn("ts_unix_trunc", from_unixtime(df.ts_new))


df.printSchema()
df.head()

root
 |-- artist: string (nullable = true)
 |-- auth: string (nullable = true)
 |-- firstName: string (nullable = true)
 |-- gender: string (nullable = true)
 |-- itemInSession: long (nullable = true)
 |-- lastName: string (nullable = true)
 |-- length: double (nullable = true)
 |-- level: string (nullable = true)
 |-- location: string (nullable = true)
 |-- method: string (nullable = true)
 |-- page: string (nullable = true)
 |-- registration: long (nullable = true)
 |-- sessionId: long (nullable = true)
 |-- song: string (nullable = true)
 |-- status: long (nullable = true)
 |-- ts: long (nullable = true)
 |-- userAgent: string (nullable = true)
 |-- userId: string (nullable = true)
 |-- ts_new: string (nullable = true)
 |-- ts_new_2: timestamp (nullable = true)
 |-- ts_unix: string (nullable = true)
 |-- ts_unix_trunc: string (nullable = true)



Row(artist='Martha Tilston', auth='Logged In', firstName='Colin', gender='M', itemInSession=50, lastName='Freeman', length=277.89016, level='paid', location='Bakersfield, CA', method='PUT', page='NextSong', registration=1538173362000, sessionId=29, song='Rockpools', status=200, ts=1538352117000, userAgent='Mozilla/5.0 (Windows NT 6.1; WOW64; rv:31.0) Gecko/20100101 Firefox/31.0', userId='30', ts_new='1538352117', ts_new_2=None, ts_unix='+50718-06-10 10:30:00', ts_unix_trunc='2018-10-01 02:01:57')

In [15]:
df.select("ts").dropDuplicates().sort("ts", ascending=False).take(10)

[Row(ts=1543799476000),
 Row(ts=1543622579000),
 Row(ts=1543622432000),
 Row(ts=1543622411000),
 Row(ts=1543622398000),
 Row(ts=1543622395000),
 Row(ts=1543622365000),
 Row(ts=1543622355000),
 Row(ts=1543622320000),
 Row(ts=1543622287000)]

In [127]:
df = df.withColumn("hour", get_hour(df.ts))
# df = df.withColumn("date", to_date("ts"))
df = df.withColumn("ts_new", to_timestamp("ts"))

In [150]:

# df = df.withColumn("ts_new", to_timestamp(df["ts"]))

# df = df.withColumn("ts_new", from_unixtime("ts"))
# get_date = udf(lambda x: from_unixtime(x / 1000.0))
get_date = udf(lambda x: (x / 1000.0), TimestampType())
# get_date = udf(lambda x: (x / 1000.0), returnType=LongType())


# df = df.withColumn("ts_new", df.ts)
df = df.withColumn("ts_new", get_date(df.ts))

df.head()
df.select(["ts", "ts_new"]).head()

PythonException: 
  An exception was thrown from the Python worker. Please see the stack trace below.
Traceback (most recent call last):
  File "c:\Users\q514907\Anaconda3\envs\spark_env\lib\site-packages\pyspark\python\lib\pyspark.zip\pyspark\worker.py", line 1247, in main
  File "c:\Users\q514907\Anaconda3\envs\spark_env\lib\site-packages\pyspark\python\lib\pyspark.zip\pyspark\worker.py", line 1239, in process
  File "c:\Users\q514907\Anaconda3\envs\spark_env\lib\site-packages\pyspark\python\lib\pyspark.zip\pyspark\serializers.py", line 225, in dump_stream
    self.serializer.dump_stream(self._batched(iterator), stream)
  File "c:\Users\q514907\Anaconda3\envs\spark_env\lib\site-packages\pyspark\python\lib\pyspark.zip\pyspark\serializers.py", line 146, in dump_stream
    for obj in iterator:
  File "c:\Users\q514907\Anaconda3\envs\spark_env\lib\site-packages\pyspark\python\lib\pyspark.zip\pyspark\serializers.py", line 214, in _batched
    for item in iterator:
  File "c:\Users\q514907

In [134]:
df.head()

# df.select("date").dropDuplicates().sort("date").show()


Row(artist='Martha Tilston', auth='Logged In', firstName='Colin', gender='M', itemInSession=50, lastName='Freeman', length=277.89016, level='paid', location='Bakersfield, CA', method='PUT', page='NextSong', registration=1538173362000, sessionId=29, song='Rockpools', status=200, ts=1538352117000, userAgent='Mozilla/5.0 (Windows NT 6.1; WOW64; rv:31.0) Gecko/20100101 Firefox/31.0', userId='30', ts_new=1538352117000)

# Exploratory Data Analysis
When you're working with the full dataset, perform EDA by loading a small subset of the data and doing basic manipulations within Spark. In this workspace, you are already provided a small subset of data you can explore.

### Define Churn

Once you've done some preliminary analysis, create a column `Churn` to use as the label for your model. I suggest using the `Cancellation Confirmation` events to define your churn, which happen for both paid and free users. As a bonus task, you can also look into the `Downgrade` events.

### Explore Data
Once you've defined churn, perform some exploratory data analysis to observe the behavior for users who stayed vs users who churned. You can start by exploring aggregates on these two groups of users, observing how much of a specific action they experienced per a certain time unit or number of songs played.

# Feature Engineering
Once you've familiarized yourself with the data, build out the features you find promising to train your model on. To work with the full dataset, you can follow the following steps.
- Write a script to extract the necessary features from the smaller subset of data
- Ensure that your script is scalable, using the best practices discussed in Lesson 3
- Try your script on the full data set, debugging your script if necessary

If you are working in the classroom workspace, you can just extract features based on the small subset of data contained here. Be sure to transfer over this work to the larger dataset when you work on your Spark cluster.

# Modeling
Split the full dataset into train, test, and validation sets. Test out several of the machine learning methods you learned. Evaluate the accuracy of the various models, tuning parameters as necessary. Determine your winning model based on test accuracy and report results on the validation set. Since the churned users are a fairly small subset, I suggest using F1 score as the metric to optimize.

# Final Steps
Clean up your code, adding comments and renaming variables to make the code easier to read and maintain. Refer to the Spark Project Overview page and Data Scientist Capstone Project Rubric to make sure you are including all components of the capstone project and meet all expectations. Remember, this includes thorough documentation in a README file in a Github repository, as well as a web app or blog post.